# Input MQTT

Reads from MQTT topics in batches or streams. Per default stores messages in buffer (in-memory, not persistent) and returns messages as JSON on HTTP endpoint (which empties the buffer). If forward_url is set messages are pushed to downstream endpoint via POST and on success buffer is emptied.

JSON format:
[{"mqtt_receive_timestamp": .., "mqtt_receive_topic": .., mqtt_payload: {..}},...]

In [ ]:
import os
#os.environ['create_image']='True'

In [ ]:
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM registry.access.redhat.com/ubi8/python-39
    RUN pip install ipython nbformat paho-mqtt flask
    ADD input-mqtt.ipynb /
    ENTRYPOINT ["ipython","/input-mqtt.ipynb","> /tmp/component.log","2> /tmp/component.err"]
    """
    with open("Dockerfile", "w") as text_file:
        text_file.write(docker_file)

    !docker build -t claimed-input-mqtt:0.2 .
    !docker tag claimed-input-mqtt:0.2 romeokienzler/claimed-input-mqtt:0.2
    !docker push romeokienzler/claimed-input-mqtt:0.2
else:
    !pip install paho-mqtt flask
    None

In [ ]:
import logging
import os
import re
import sys
import random
from paho.mqtt import client as mqtt_client
import threading
from flask import Flask
from flask import abort
import queue
import time
import json
from threading import Thread
import requests 

In [ ]:
# mqtt port
port = int(os.environ.get('port',1883))

# mqtt server
broker = os.environ.get('broker')

# mqtt topics, format 'topic1,...'
topics = os.environ.get('topics', '')

# mqtt user name
username = os.environ.get('username', '')

# mqtt password
password = os.environ.get('password', '')

# forward messages to forward_url via POST
forward_url = os.environ.get('forward_url')

In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)
    
port = int(port)
topics = [(topic.strip(),0) for topic in topics.split(',')] if topics != '' else []

In [ ]:
def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)
    # Set Connecting Client ID
    client = mqtt_client.Client(f'python-mqtt-{random.randint(0, 1000)}')
    client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client

In [ ]:
client = connect_mqtt()
q = queue.Queue()
client.subscribe(topics)
#client.on_message = lambda client, userdata, msg: q.put(()),json.dumps(msg.payload.decode())))
client.on_message = lambda client, userdata, msg: q.put('{"mqtt_receive_timestamp": '+
                                                        str(int(round(time.time() * 1000)))+
                                                                  ', "mqtt_receive_topic":  "'+
                                                                  msg.topic+
                                                                  '", "mqtt_payload": '+ 
                                                                  msg.payload.decode()+
                                                        '}'
                                                                 )


In [ ]:
x = threading.Thread(target=client.loop_forever)
x.start()

In [ ]:
def queue_as_json():
    return_list = []
    try:
        while True:
            message = q.get(block=False)
            print(message)
            return_list.append(message)
    except queue.Empty:
        print('empty queue!')
    return "["+(",".join(return_list))+"]"

In [ ]:
def push_downstream(forward_url,queue_as_json):
    while True:
        message = queue_as_json()
        if len(message)>2:
            requests.post(forward_url, data=message)
        time.sleep(1)


if forward_url is not None:
    new_thread = Thread(target=push_downstream,args=(forward_url,queue_as_json))
    new_thread.start()

In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    if forward_url is None:
        return queue_as_json()
    else:
        return 'MQTT Service running'


app.run(host='0.0.0.0', port=8080)